The challenge: 

we are concerned with 7-segment displays, where each segment is controlled by another wire and the wires are named a-g

<img width=300; src="default_segments.png">

But the wires have been crossed and we have to deduce which wire (a-g) controls which part of the display.

The input is the 10 different observations and after the '|', we get 4 numbers that we have to translate.

example-input: '''acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf'''

## Part 1

How many times do digits 1, 4, 7 or 8 appear?

1 (two segments), 4 (4 segments), 7 (three segments), 8 (7 segments)

In [5]:
def get_segments(line):
    observations, outputs = [part.strip() for part in line.split('|')]
    observations = observations.split(' ')
    outputs = outputs.split(' ')
    return observations, outputs

def part1(file_name):
    with open(file_name, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    special_numbers = 0

    for line in lines:
        observations, outputs = get_segments(line)
        output_lenghts = [len(output) for output in outputs]
        special_numbers += output_lenghts.count(2)
        special_numbers += output_lenghts.count(4)
        special_numbers += output_lenghts.count(3)
        special_numbers += output_lenghts.count(7)
    
    print(special_numbers)

In [7]:
part1('input.txt')

362


## Part 2

Now deduce the mapping from signals to the observed numbers.

Then translate the 4 digit outputs, add those numbers across all lines to get the solution.

-aaaa- original wire names

b----c

b----c

-dddd-

e----f

e----f

-gggg-

In [12]:
def get_segments(line):
    observations, outputs = [part.strip() for part in line.split('|')]
    observations = observations.split(' ')
    outputs = outputs.split(' ')
    return observations, outputs

def deduct_wires(observations):
    real_to_fake_wires = dict()
    # identify the observed signals of 1, 4, 7, 8 because they are unique
    obs_one = [entry for entry in observations if len(entry) == 2][0]
    obs_four = [entry for entry in observations if len(entry) == 4][0]
    obs_seven = [entry for entry in observations if len(entry) == 3][0]
    obs_eight = [entry for entry in observations if len(entry) == 7][0]
    # grouping the observations with length 5, corresponding to numbers 2, 3, 5
    obs_five_segments = [entry for entry in observations if len(entry) == 5]
    # grouping the observations with length 6, corresponding to numbers 6, 9, 0
    obs_six_segments = [entry for entry in observations if len(entry) == 6]

    # finding wire 'a' by taking the wire that is in 7 but not in 1
    real_to_fake_wires['a'] = [wire for wire in obs_seven if wire not in obs_one][0]
    # finding wire 'd' by taking the wire from 4 that is in all observations with length 5
    real_to_fake_wires['d'] = [wire for wire in obs_four if all([wire in fiver for fiver in obs_five_segments])][0]
    # finding wire 'b' by taking the wire from 4 without the ones in 1 and fake_d
    real_to_fake_wires['b'] = (set(obs_four) - set(obs_one).union(set(real_to_fake_wires['d']))).pop()
    # finding wire 'f' by taking the entry from 1 that is in all observations with length 6
    real_to_fake_wires['f'] = [wire for wire in obs_one if all([wire in sixer for sixer in obs_six_segments])][0]
    # the other entry in 1 is then 'c'
    real_to_fake_wires['c'] = (set(obs_one) - set(real_to_fake_wires['f'])).pop()
    # identify the two unmapped mystery wires
    mystery_wires = list(set("abcdefg") - set(real_to_fake_wires.values()))
    # the mystery wire that only appears in four different numbers is 'e'
    mystery_wire_counts = [len([obs for obs in observations if mystery_wire in set(obs)]) for mystery_wire in mystery_wires]
    real_to_fake_wires['e'] = mystery_wires[0] if mystery_wire_counts[0]==4 else mystery_wires[1]
    # the other one is 'g'
    real_to_fake_wires['g'] = mystery_wires[1] if mystery_wire_counts[0]==4 else mystery_wires[0]

    # reverse dict
    fake_to_real_wires = {v: k for k, v in real_to_fake_wires.items()}
    return fake_to_real_wires

def translate_signal(signal, fake_to_real_wires):
    real_signal = ''
    for wire in signal:
        real_signal += fake_to_real_wires[wire]
    return real_signal

def signal_to_number(signal):
    sorted_signal = ''.join(sorted(signal))
    signal_mapping = {'cf': '1',
                        'acdeg': '2',
                        'acdfg': '3',
                        'bcdf': '4',
                        'abdfg': '5',
                        'abdefg': '6',
                        'acf': '7',
                        'abcdefg': '8',
                        'abcdfg': '9',
                        'abcefg': '0'}
    return signal_mapping[sorted_signal]

def part2(file_name):
    with open(file_name, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    output_sum = 0

    for line in lines:
        observations, outputs = get_segments(line)
        fake_to_real_wires = deduct_wires(observations)
        translated_outputs = [signal_to_number(translate_signal(output, fake_to_real_wires)) for output in outputs]
        output_sum += int(''.join(translated_outputs))
    print(output_sum)

In [13]:
part2('example.txt')

61229


In [14]:
part2('input.txt')

1020159
